# Notebook for extracting drought indicators

This notebook will extract various drought indicators from remote sensing data using Google Earth Engine (GEE) and visualise them using the geemap python package. \\
 \\
### Indicators to be calculated:
- Vegetation Condition Index (VCI)
- Vegetation Health Index (VHI)
- Temperature Condition Index (TCI)

### Data Sources:
- MODIS
- Landsat
- Sentinel-2

MODIS is the best starting point as Normalised Difference Vegetation Index (NDVI) and Land Surface Temperature (LST) already exist, making calculation of the other indicators straightforward. \\
 \\
Landsat and Sentinal-2 will be used but they do not have NDVI and LST as products and so they will have to be calculated from individual bands within the data collections. \\
 \\
All the data sources required exist within Google Earth Engine already and so can be imported quickly and easily without the need for downloading any large-scale gridded datasets.


## Useful resources

[GEE Code Editor](https://code.earthengine.google.com/): Built in documentation which can be filtered to find methods of interest. Found under the Docs tab.

[Official GEE guides](https://developers.google.com/earth-engine/guides): Outlines GEE functionality but most of the example code is in javascript.

[geemap tutorials](https://geemap.org/tutorials/): A full list of tutorials from the geemap developer can be found with links to videos and notebooks.

[GEE Courses](https://courses.geemap.org/): The geemap developer has created tutorials for using GEE in python since most of the official documentation uses javascript.

## Setting up the Environment

We need to import the desired packages and then connect to GEE \\
 \\
geemap is not installed in colab by default so this needs to be installed every time we connect to a runtime \\
 \\
ee is the package for connecting to Google Earth Engine and accessing GEE functionality, geemap is the package for easy interactive Earth Engine visualisations

In [ ]:
# first import the required packages
!pip install -q geemap

import ee
import geemap
import os

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 KB 14.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.6/99.6 KB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 KB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 KB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 KB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 KB 2.7 MB/s eta 0:00:00
 

In order to connect to Google Earth Engine we need to authorize GEE to connect to the notebook.

In [ ]:
ee.Authenticate()

ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=piTBlgBhM8yh7av9kyElsOvot9pRkr8NjaoHFGTYzL0&tc=2JUw8aHAMGx6kZALM7nQ03ospioKdmGpcwCtVTJlzYw&cc=me2YZinL_RLCgdetEFWausVIUR4BbJDTN4eesOJGHWg

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AWtgzh75xI5BHLPlRXbczb1vA1Z1ObY-GTUc59DIlhwLpNL_GdnWmSTtdYQ

Successfully saved authorization token.


## Defining Variables

We want to define some variables that we will use in various places throughout the script. Having them all in once place means we don't need to search through the document every time we want to change a parameter. \\
 \\
Examples of variables to use would be start and end years of the data. \\
 \\
Before calling the data we need to define a region of interest (roi) that we want to focus on. In this case we are using a rectangle that covers the UK (between -8.5 and 2 longitude and between 49 and 61 latitude). We want to crop to this smaller area since we aren't interested in data elsewhere and a smaller dataset allows for faster processing and smaller output files. This region of interest will also be used for producing the animation later.

In [ ]:
# define start and end year
# note that for the current version of the code these need to be complete years
#   ie every month in the year needs to have data
start_year = 2001
end_year = 2022

# define region of interest
roi = ee.Geometry.Polygon([[[-8.5, 61], [-8.5, 49], [2, 49], [2, 61]]], None,False)

## Accessing the Data

Now we are connected to GEE we can begin processing the data. \\
 \\
GEE makes loading data straightforward as there is a large library of datasets that are built into GEE. \\
 \\
We can search for the datasets of interest in the [GEE Code Editor](https://code.earthengine.google.com/) \\
 \\
Finding the datasets here will give us the snippet required to load the collections and the method for loading them in is shown below. Note the use of the ".select(' ')" statement to pick out the variable of interest from the dataset. The variables can be found under the "bands" tab when reading the description of a dataset. \\
 \\
**An important note on terminology:** \\
Google Earth Engine makes use of the terms "image" and "image collection" to refer to data and so when referring to data within the code we will be using these terms. An image is a single time-slice of a gridded dataset and can consist of multiple "bands" (these are variables in classic datasets). Image collections are groups of images and, for the purposes of this document, each image will represent a different time step. More information about Earth Engine data structures can be found in the [Get Started section of the GEE Guide](https://developers.google.com/earth-engine/guides/getstarted#earth-engine-data-structures). \\
 \\
**Back to the code:** \\
Once the desired variable has been selected we can clip the image collection to the roi defined above. Operations cannot be performed on a whole image collection, only individual images, so we need to map operations to all images in a collection. More information about mapping to over an image collection can be found in the [Mapping section of the GEE Guide](https://developers.google.com/earth-engine/guides/ic_mapping). \\
 \\
Throughout the document we will make use of lambda functions, particularly when mapping a function across an image collection. Lambda functions work identically to regular functions but are on a single line, are typically only used in a single place, and can't contain conditional statements. Lambda functions have a simple format, for example here are two lambda functions. 1) Doubling the input variable, 2) Multiplying two input variables together: 

```
1) lambda x: x*2
2) lambda x,y : x*y
```
In the code snippet below, the lambda function clips each image to the region of interest and this function is mapped across all the images in the collection. \\
 \\
Also below, the commented line that makes use of ".getInfo()" can be used to sense-check the properties of the image collection. This is particularly useful when we want to ensure we are getting the correct number of images as an output.


In [ ]:
# read in the MODIS NDVI data
MODIS_NDVI_raw = ee.ImageCollection("MODIS/061/MOD13A2").select('NDVI')

MODIS_NDVI = MODIS_NDVI_raw.map(lambda image: image.clip(roi))

#MODIS_NDVI.getInfo()

## Formatting the data



The MODIS data is at 8-day and 16-day timesteps for the LST and NDVI respectively. This means we need to convert the data to be monthly and this can be done with nested functions. \\
 \\
The function below takes in an image collection as the argument. \\
Next it defines sequences of months and years that will be used for calculating the monthly averages. \\
The function ***yearly*** takes in a year as an argument and the function ***monthly*** takes in a month as an argument. \\
The sequences that we defined earlier are then mapped to these functions. \\
 \\
***col2monthly*** maps the years to ***yearly*** and returns an image collection from the results. \\
***yearly*** maps the months to ***monthly*** \\
***monthly*** applies the filters and takes the mean, returning the resulting image


In [ ]:
def col2monthly(col):
    
    # define sequences of months and years
    months = ee.List.sequence(1, 12)
    years = ee.List.sequence(start_year,end_year)

    # function to map years to
    def yearly(y):
     
      # function to map months to
      def monthly(m):
      
        # filter the image collection to the desired year and month
        # take a mean of the filtered images
        # set new properties for the month, year, and unique ID
        # return this new image
        return (col.filter(ee.Filter.calendarRange(y, y, 'year'))
                .filter(ee.Filter.calendarRange(m, m, 'month')).mean()
                .set('month', m)
                .set('year', y)
                #.set('ID', ee.Number(m).format('%02d'))
                )
      # apply the monthly function to all the months for the given year
      return months.map(monthly)

    # create a collection of images from the output of the monthly function
    # the yearly function is applied to all years
    col_m = ee.ImageCollection.fromImages(years.map(yearly).flatten())

    return col_m

Once the function has been set up we can simply call the function whenever we have an image collection that we wish to convert into a monthly time series. \\
 \\
In the case of MODIS NDVI we need to apply an additional calculation to the monthly data since the raw MODIS NDVI data has a scale factor. Information about the units and scale factors can be found under the "bands" tab of the window that appears when you select a dataset from the search bar in the Earth Engine code editor. \\
 \\
When performing operations such as multiplications or subtractions, GEE does not assume that properties of the image are still correct since not all operations will preserve properties. We want to maintain our 'month' and 'year' properties though so we have to add an additional segment to copy the properties from the old image to the new one.

In [ ]:
# extract monthly image collection from 16-day MODIS NDVI image collection
NDVI_Monthly = col2monthly(MODIS_NDVI)

# multiply by 0.0001 and copy the image properties
NDVI_Monthly = NDVI_Monthly.map(lambda image: image.multiply(0.0001).copyProperties(image, image.propertyNames()))

#NDVI_Monthly.getInfo()

Scale factors and conversions between units are things to be aware of when processing datasets. In the case of Land Surface Temperature, the value may be in Kelvin and so if we wanted to convert this to celsius we would need to apply the scale factor and then subtract 273.15 from the original value.

## Calculating VCI

Now we have a monthly image collection for NDVI which has had the correct scale factors applied, giving us the true values for this variable. \\
 \\
The next step is to calculate a drought index. \\
 \\
VCI is derived from NDVI and compares the current value to the long term maxima and minima for the given calendar month: \\
$$ \text{VCI} = \frac{x_{i,j} - \text{min}(x_{i})}{ \text{max}(x_{i}) - \text{min}(x_{i}) } \times 100$$ 
Where $x$ is NDVI, $i$ is a given calendar month (1-12), and $j$ is a given year (2001-2022). If we want to calculate the NDVI for January 2012 the values are as follows: \\
- $ x_{i,j} = $ Mean NDVI for January 2012 \\
- $\text{min}(x_{i})$ = Minimum NDVI for January of any year \\
- $\text{max}(x_{i})$ = Maximum NDVI for January of any year

From this we can piece together the additional data required. We have calculated all our $x_{i,j}$ values in the NDVI_Monthly image collection so now we need the max and min values. Thankfully this can be straightforward as we have a function that we can modify to calculate max and min for given calendar months. \\
 \\
The functions below are very similar to the monthly means function above but they are simplified. In this case we do not need to map both the years and the months, we only need to map the months. This will give us an output image collection with 12 images in, one for each month. The two functions are nearly identical with the only difference being whether it calculates maximum or minimum for the month.

In [ ]:
def min_monthly(col):
    
    # define sequence of months
    months = ee.List.sequence(1, 12)
    
    # function to map months to
    def monthly(m):
      
      # filter the image collection to the desired month
      # take a min of the filtered images
      # set new properties for the month and unique ID
      # return this new image
      return (col.filter(ee.Filter.calendarRange(m, m, 'month')).min()
              .set('month', m)
              .set('ID', ee.Number(m).format('%02d'))
              )

    # create a collection of images from the output of the monthly function
    # the monthly function is applied to all months
    col_min = ee.ImageCollection.fromImages(months.map(monthly).flatten())

    return col_min


def max_monthly(col):
    
    # define sequence of months
    months = ee.List.sequence(1, 12)
    
    # function to map months to
    def monthly(m):
      
      # filter the image collection to the desired month
      # take a max of the filtered images
      # set new properties for the month and unique ID
      # return this new image
      return (col.filter(ee.Filter.calendarRange(m, m, 'month')).max()
              .set('month', m)
              .set('ID', ee.Number(m).format('%02d'))
              )

    # create a collection of images from the output of the monthly function
    # the monthly function is applied to all months
    col_max = ee.ImageCollection.fromImages(months.map(monthly).flatten())

    return col_max

We can now use the above functions to get image collections for the max and min NDVI for each month. Since these functions take in the original MODIS NDVI data we again need to apply the scale factor to the output image collections and preserve the properties of each image.

In [ ]:
NDVI_mon_min = min_monthly(MODIS_NDVI)
NDVI_mon_min = NDVI_mon_min.map(lambda image: image.multiply(0.0001).copyProperties(image, image.propertyNames()))

NDVI_mon_max = max_monthly(MODIS_NDVI)
NDVI_mon_max = NDVI_mon_max.map(lambda image: image.multiply(0.0001).copyProperties(image, image.propertyNames()))

#NDVI_mon_max.getInfo()

The next step is to apply the desired calculation to the image collections. This manipulation once again needs to be done through mapping. \\
 \\
Like the col2monthly function earlier, we can set up a function to work with both the months and the years to produce a new image collection containing the VCI values. In this case the function will take 3 inputs rather than 1. 
- Monthly mean NDVI
- Calendar Month max NDVI
- Calendar Month min NDVI

In [ ]:
def vci_calc(col, max_vals, min_vals):
    
    # define sequences of months and years
    months = ee.List.sequence(1, 12)
    years = ee.List.sequence(start_year,end_year)

    # function to map years to
    def year_map(y):
     
      # function to map months to
      def month_map(m):
      
        # filter the image collections to the desired year and month
        # the outputs of the filters will be image collections with a single image in each
        # apply a mean to each collection to convert to an image (this won't alter the values since the collections only have a single image in)
        # select the NDVI band from each of these images and perform the VCI calculation
        # set new properties for month and year
        # return this new image

        x = col.filter(ee.Filter.eq('year',y)).filter(ee.Filter.eq('month',m)).mean()
        max_x = max_vals.filter(ee.Filter.eq('month',m)).mean()
        min_x = min_vals.filter(ee.Filter.eq('month',m)).mean()

        return (  ( x.select('NDVI').subtract(min_x.select('NDVI')) )
                .divide( max_x.select('NDVI').subtract(min_x.select('NDVI')) )
                .multiply(100)
                .set('month', m)
                .set('year', y)  )
        
      # apply the filter to all the months for the given year
      return months.map(month_map)

    # create a collection of images from the output of the monthly function
    # the monthly function is applied to all years
    col_m = ee.ImageCollection.fromImages(years.map(year_map).flatten())

    return col_m

We can feed our derived NDVI image collections into the above function to generate a time series of VCI values. We can also use the select function to rename the NDVI band to VCI to accurately describe the data in the image collection.

In [ ]:
vci = vci_calc(NDVI_Monthly, NDVI_mon_max, NDVI_mon_min)

vci = vci.select(['NDVI'], ['VCI'])

#vci.getInfo()

## Visualising VCI



Now we have an image collection containing VCI values. We can put this onto an interactive map to visualise the VCI. In order to do this we need to convert the image collection into a single image which can be done using the '.toBands()' function. This converts each image in a collection to an individual band (or variable) in a single image. By default this would give the bands the names '0_VCI', '1_VCI' etc. so we can once again use the select function to rename these bands. \\
 \\
The create_band_names() function below simply creates a list of band names with the format 'mm-yyyy_VCI' to give them more meaning for plotting. We can get the original band names with the '.bandNames()' function.

In [ ]:
# convert image collection to a single image
vci_img = vci.toBands()

# extract band names from the image
original_names = vci_img.bandNames()

# define a function to create a list of new band names
# the .rjust is there to turn months like January from '1' to '01'
def create_band_names():
    months = range(1, 13)
    years = range(start_year,end_year+1)
    names = []
    for year in years:
        for month in months:
            names.append(str(month).rjust(2,'0')+'-'+str(year)+'_VCI')

    return(names)

# get the list of new band names
new_names = create_band_names()

# rename the bands in the image
vci_img = vci_img.select(original_names, new_names)

We can use geemap to create an interactive map window to explore the data in detail. This is done by setting up a basic map and adding the VCI image as a layer. The .addLayer function here takes 3 inputs: \\
The first is the image we want to add. \\
The second is a dictionary of visualisation parameters (in this case we don't have any since we will use the interface to change these). \\
The third is the label we want to give this layer. \\
 \\
To get the map to look sensible some changes to settings are required and this can be done as follows:
- Click the "toolbar" button in the top right of the map window
- Click the "layers" button that has appear to the left of the "toolbar" button
- Click the settings button next to VCI (this can take a few seconds to load)
- From here we can change what we display. The clearest for this type of image is "1 band (Grayscale)"
- We can use the drop down box to select which band (timestep) we want to view
- The range defaults to 0-100 which is perfect for VCI so doesn't need changing
- We can choose whichever colormap we want, RdYlGn is sensible for VCI since this is a red-green scale with red being low VCI and green being high.
- Once we are happy we can hit apply, wait a few seconds, and the VCI will be shown on the map

Changes can be made to the map properties at any time through this interactive map interface. \\
 \\
Examples of colormaps can be found in the [MatPlotLib Documentation](https://matplotlib.org/stable/gallery/color/colormap_reference.html).

In [ ]:
Map = geemap.Map()
Map.addLayer(vci_img, {}, 'VCI')
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

If we want we can also produce a gif showing the changes in VCI for the UK. By changing the variable y we can select which year to put into the gif which is done using a filter. We also define text to label the gif and these can be changed to any sensible labels we want. \\
 \\
The fps variable defines the number of frames per second we want for the gif, this is used later to ensure the labelled version runs at the same speed. \\
 \\
We set up an area of interest (aoi) for the gif (in this case we use the roi defined at the start) and then define some basic video arguments. \\
 \\
A path for the vci gif is defined, this puts it into a temporary google colab file directory (also known as session storage). All files in this temporary storage can be seen by clicking the files button to the left of the screen underneath the {x} symbol. \\
 \\
The developer of the geemap package has created a full [video tutorial](https://www.youtube.com/watch?v=fDnDVuM_Ke4) and accompanying [jupyter notebook](https://github.com/giswqs/geemap/blob/master/examples/notebooks/16_add_animated_text.ipynb) for adding animated text to a gif. This tutorial was followed when developing the below code for the generation of an annotated gif showing the evolution of VCI in the UK for a given year.

In [ ]:
# define year to animate
y=2022

# define text labels for animation 
text = [str(n).zfill(2) + "-" + str(y) for n in range(1, 13)]
#text = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

# define number of frames per second of video
fps = 2


# filter the VCI to the desired year
video_vci = vci.filter(ee.Filter.eq('year',y))

# define area of interest for the gif
aoi = roi
#ee.Geometry.Polygon(
#  [[[-8.5, 61], [-8.5, 49], [2, 49], [2, 61]]], None,
#  False)

# define video arguments
videoArgs = {"dimensions": 800, 
             "region": aoi, 
             "framesPerSecond": fps, 
             "crs": 'EPSG:3857', 
             "min": 0, 
             "max": 100, 
             "palette": ['red', 'yellow', 'green']}

# print statement can be used to quickly view the unlabelled gif
#print(video_vci.getVideoThumbURL(videoArgs))

# save gif to the session storage
saved_gif = os.path.join('vci.gif')
geemap.download_ee_video(video_vci, videoArgs, saved_gif)

Generating URL...
Please wait ...
The GIF image has been saved to: /content/vci.gif


We define a path for the labelled version of the gif, define the duration of each frame based off the fps we set earlier. The gif we just saved gets fed into the '.add_text_to_gif' function and a new gif is saved to the session storage with the desired text labels applied to it. \\
 \\
If we want to download this gif we can go to the session storage files tab, find the correct file, hover over it, press the three dots that appear, and download it from there. It can also be viewed in colab with geemap.show_image()

In [ ]:
out_gif = os.path.join('vci_labelled.gif')

# define duration of each frame based on the FPS (1000ms = 1 second)
dur = 1000/fps

# call the function to add text to the gif
geemap.add_text_to_gif(
    saved_gif,
    out_gif,
    xy=('3%', '5%'),
    text_sequence=text,
    font_size=30,
    font_color='#ffffff',
    duration = dur,
)

# show the labelled gif in colab
geemap.show_image(out_gif)

Output()